In [9]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
import json

In [10]:
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))
container_client = blob_service_client.get_container_client(os.getenv("AZURE_CONTAINER"))

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [22]:
staging_dir = '/staging'
os.makedirs(staging_dir, exist_ok=True)


blob_paths = [
'BUS5000/Introduction/10dayMBA - Intro.pdf',
'BUS5000/Introduction/STUDENT_NOTES/s1/s1w0.docx',
'BUS5000/Introduction/STUDENT_NOTES/s2/s2w0.docx'
]
# List to store file objects
uploaded_files = []

for blob_path in blob_paths:
    # Adjust the path to save in the staging directory
    staging_path = os.path.join(staging_dir, os.path.basename(blob_path))

    # Download the file from Azure Blob
    blob_client = container_client.get_blob_client(blob_path)
    with open(staging_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

    # Upload the file to OpenAI
    with open(staging_path, "rb") as file:
        response = client.files.create(file=file, purpose="assistants")
        uploaded_files.append(response)

    # Delete the file from the staging directory
    os.remove(staging_path)

# Function to convert FileObject to a serializable dictionary
def file_object_to_dict(file_obj):
    return {
        'id': file_obj.id,
        'bytes': file_obj.bytes,
        'created_at': file_obj.created_at,
        'filename': file_obj.filename,
        'object': file_obj.object,
        'purpose': file_obj.purpose,
        'status': file_obj.status,
        'status_details': file_obj.status_details
    }

# Convert each FileObject in the list to a dictionary
file_dicts = [file_object_to_dict(file_obj) for file_obj in uploaded_files]

# Save the list of dictionaries as a JSON list
json_path = './.bin/files.json'
with open(json_path, 'w') as json_file:
    json.dump(file_dicts, json_file)

In [19]:
print(client.files.list())

SyncPage[FileObject](data=[], object='list', has_more=False)


In [21]:
def delete_files_from_openai():
    json_path = './.bin/files.json'
    # Load the file data from the JSON file
    with open(json_path, 'r') as json_file:
        file_data = json.load(json_file)

    # Iterate through each file in the list
    for file_info in file_data:
        file_id = file_info['id']
        # Delete the file using OpenAI API and print the status
        st.session_state.ai_client.files.delete(file_id)
    
    # Open the file in write mode and close it to clear its contents
    with open(json_path, 'w') as file:
        pass  # Opening in 'w' mode and closing will clear the file

delete_files_from_json()

In [26]:
json_path = './.bin/files.json'

fileids = []

# Load the file data from the JSON file
with open(json_path, 'r') as json_file:
    file_data = json.load(json_file)
        # Iterate through each file in the list
    for file_info in file_data:
        file_id = file_info['id']
        fileids.append(file_id)
    
print(fileids)

['file-1I7bhX1YyIhKCJ7ddouuPy60', 'file-oxcDEGJLNutupSASmnV2vM3w', 'file-fQZCpJfuH5J6TQ6O6Qpt3Zxn']
